### set up parameters & grid

In [1]:
import numpy as np
from landlab import RasterModelGrid
from landlab import CLOSED_BOUNDARY, FIXED_VALUE_BOUNDARY
from landlab.components import FlowRouter
from landlab.components import LinearDiffuser
from landlab.components import StreamPowerEroder
from landlab.components import drainage_density
from landlab.components import SedDepEroder
from landlab.io.netcdf import write_netcdf
from landlab.io.netcdf.write import _add_time_variable
#from landlab.components.flow_routing import lake_mapper, DepressionFinderAndRouter
from landlab import imshow_grid
from matplotlib import pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
import time
%matplotlib inline

In [2]:
#Grid
ncols = 301
nrows = 301
dx    = 100   #m
vegi_trace_x = 75

#timesteps
out_int  = 100
total_T1 = 10e5       #yrs
dt = 100                #yrs
timeVec = np.arange(0,total_T1,dt) #for plotting purposes
nt = total_T1/dt
no = total_T1/out_int
zp = len(str(int(no)))
#zp = len(str(int(nt)))
#uplift
uplift_rate = 1e-1 #m/yr
uplift_per_step = uplift_rate * dt

#eroder/diffuser
Ksp = 1e-5
msp = 0.6
nsp = 0.8
linear_diffusivity_base = 1e-2 #m^2/s

#critical_area
crit_area = 10e6
#fluvial threshold
thresh_sp = 2.0

#time
elapsed_time = 0

In [3]:
mg = RasterModelGrid((nrows,ncols),dx)
z = mg.add_ones('node','topographic__elevation',)
#Add some initial roughness to the landscape and some initial water-depth 
initial_roughness = np.random.rand(z.size)/1000
z += initial_roughness 

### set up boundary - conditions

In [4]:
for edge in (mg.nodes_at_left_edge,mg.nodes_at_right_edge, mg.nodes_at_top_edge):
    mg.status_at_node[edge] = CLOSED_BOUNDARY
for edge in (mg.nodes_at_bottom_edge):
    mg.status_at_node[edge] = FIXED_VALUE_BOUNDARY

### create vegetation field

In [5]:
vegi_perc = mg.zeros('node',dtype=float)
##THIS CREATES A DEFINED VEGETATION FIELD WITH A FIXED BOUNDARY
#sp_t = mg.ones('node',dtype=float)
#vegi_trace_x = 200
#more_vegi = np.where(mg.x_of_node >= vegi_trace_x)
#less_vegi = np.where(mg.x_of_node < vegi_trace_x)
#vegi_perc[less_vegi] = 0.1
#vegi_perc[more_vegi] = 0.7
##THIS CREATES A RANDOMIZED VEGETATION FIELD
#vegi_perc = np.random.rand(z.size)

##THIS CREATES A SIN-SHAPED VEGETATION TIME-SERIES
#Sin function between 0 and 1
vegi_test_timeseries = (np.sin(0.00015*timeVec)+1)/2 
#Sin function between 0.2 and 0.4
#vegi_test_timeseries = (np.sin(0.00015*timeVec)+ 2) / 9
#Sin function between 0.8 and 1.0
#vegi_test_timeseries = (np.sin(0.00015*timeVec)+ 9) / 10
#imshow_grid(mg,vegi_perc)

### Create routine for K-field modulation

In [6]:
AqDens = 1000.0 #Density of Water [Kg/m^3]
grav   = 9.81   #Gravitational Acceleration [N/Kg]
n_soil = 0.025  #Mannings roughness for soil [-]
n_VRef = 0.2    #Mannings Reference Roughness for Vegi [-]
v_ref  = 0.8    #Reference Vegetation Density
w      = 1.    #Some scaling factor for vegetation [-?]

#Calculations after Istanbulluoglu
nSoil_to_15 = np.power(n_soil, 1.5)
Ford = AqDens * grav * nSoil_to_15
n_v_frac = n_soil + (n_VRef*(vegi_perc/v_ref)) #self.vd = VARIABLE!
n_v_frac_to_w = np.power(n_v_frac, w)
Prefect = np.power(n_v_frac_to_w, 0.9)

Kv = Ksp * Ford/Prefect

In [7]:
Kfield = mg.zeros('node',dtype = float)
Kfield = Kv
#Kfield[np.where(mg.x_of_node >= vegi_trace_x)] = 5e-3
#imshow_grid(mg,Kfield)

### create linear diffusivity field

In [8]:
lin_diff = mg.zeros('node', dtype = float)
lin_diff = linear_diffusivity_base*np.exp(-vegi_perc)
#lin_diff[np.where(mg.x_of_node >= vegi_trace_x)] = 0.03
#lin_diff[np.where(mg.x_of_node < vegi_trace_x)] = 0.06
#lin_diff += 0.01
#imshow_grid(mg,lin_diff)

### Create Field with values of 'threshold_sp', same spatial distribution than vegetation

In [9]:
threshold_arr = mg.zeros('node',dtype=float)
threshold_arr[np.where(mg.x_of_node <= vegi_trace_x)] += 200
threshold_arr[np.where(mg.x_of_node > vegi_trace_x)] += 200
threshold_field = mg.add_field('node','threshold_sp',threshold_arr,noclobber = False)

### initialize the erosional components

In [ ]:
fr = FlowRouter(mg)
ld = LinearDiffuser(mg,linear_diffusivity=lin_diff)
sp = StreamPowerEroder(mg, K_sp = Kfield, m_sp=msp, n_sp=nsp, sp_type = 'set_mn',threshold_sp=thresh_sp)
#df = DepressionFinderAndRouter(mg)

### run the main loop

In [ ]:
t0 = time.time()
counter = 0
#DrainageDensity
mean_dd     = []
#Mean, Fluvial, Hillslope Erosion
mean_E      = []
mean_riv_E  = []
mean_hill_E = []
#Mean, Max, Min Slope
mean_slope  = []
max_slope   = []
min_slope   = []
#Mean, Max, Min Elevation
mean_elev   = []
max_elev    = []
min_elev    = []
#Mean_DHDT
mean_dhdt   = []
dhdtArr     = []
#K values River and Diffusivity
mean_K_riv  = []
mean_K_diff = []


#second uplif rate
uplift_per_step = uplift_rate * dt

ncCounter = 0

while elapsed_time < total_T1:
        
    z0 = mg.at_node['topographic__elevation'].copy()
    
    #Actual erosion routines:
    fr.run_one_step(dt=dt)
    ld.run_one_step(dt=dt)
    sp.run_one_step(dt=dt)
    #df.map_depressions()
    
    #Add uplift
    z[mg.core_nodes] += uplift_rate * dt #add uplift
    #z[np.where(mg.x_of_node > 400)] += 2*uplift_rate * dt
    #mg.at_node['topographic__elevation'][mg.boundary_nodes] = 0
    
    #calculate channel_network and drainage density
    channel_mask = mg.at_node['drainage_area'] > crit_area
    dd = drainage_density.DrainageDensity(mg, channel__mask = channel_mask)
    mean_dd.append(dd.calc_drainage_density())
    
    
    #calculate erosion rates by dh/dt diff of matrix4
    dh = mg.at_node['topographic__elevation'] - z0
    dhdt = dh/dt
    dhdtArr.append(dhdt)
    mean_dhdt.append(np.mean(dhdt))
    mean_E.append(np.mean(uplift_rate - dhdt))
    
    #calculates only river erosion rates
    dh_riv = mg.at_node['topographic__elevation'][np.where(mg.at_node['drainage_area'] > crit_area)] - z0[np.where(mg.at_node['drainage_area'] > crit_area)]
    dhdt_riv = dh_riv/dt
    mean_riv_E.append(np.mean(uplift_rate - dhdt_riv))
    
    #calculates hillslope erosion
    dh_hill = mg.at_node['topographic__elevation'][np.where(mg.at_node['drainage_area'] <= crit_area)] - z0[np.where(mg.at_node['drainage_area'] <= crit_area)]
    dhdt_hill = dh_hill/dt
    mean_hill_E.append(np.mean(uplift_rate - dhdt_hill))
    
    #Updates the K-field after every timestep according to the vegetation-field
    vegi_perc = np.random.rand(z.size)/100
    vegi_perc += vegi_test_timeseries[int(elapsed_time/dt)-1]
    vegi_perc.clip(0.,1.)
    n_v_frac = n_soil + (n_VRef*(vegi_perc/v_ref)) #self.vd = VARIABLE!
    n_v_frac_to_w = np.power(n_v_frac, w)
    Prefect = np.power(n_v_frac_to_w, 0.9)
    Kv = Ksp * Ford/Prefect
    Kfield = Kv
    
    #initializes the stream-power-eroder new ? very slow I guess...
    sp = StreamPowerEroder(mg, K_sp = Kfield, m_sp=msp, n_sp=nsp, sp_type = 'set_mn', threshold_sp=thresh_sp)
    
    #LINEAR DIFUSSIVITY
    lin_diff = linear_diffusivity_base*np.exp(-vegi_perc)
    ld = LinearDiffuser(mg,linear_diffusivity=lin_diff)
      
    #save mean_K_diff and mean_K_riv
    mean_K_riv.append(np.mean(Kfield))
    mean_K_diff.append(np.mean(lin_diff))
    
    #save mean, max, min slope
    mean_slope.append(np.mean(mg.at_node['topographic__steepest_slope'][mg.core_nodes]))
    max_slope.append(np.max(mg.at_node['topographic__steepest_slope'][mg.core_nodes]))
    min_slope.append(np.min(mg.at_node['topographic__steepest_slope'][mg.core_nodes]))
    
    #save mean, max, min elev
    mean_elev.append(np.mean(mg.at_node['topographic__elevation'][mg.core_nodes]))
    max_elev.append(np.max(mg.at_node['topographic__elevation'][mg.core_nodes]))
    min_elev.append(np.min(mg.at_node['topographic__elevation'][mg.core_nodes]))
    
    
    #Output every 100 years
    if elapsed_time % out_int == 0:
        print('Elapsed Time:' , elapsed_time,'writing output!')
        plt.figure()
        imshow_grid(mg,'topographic__elevation',grid_units=['km','km'],var_name = 'Elevation',cmap='terrain')
        plt.savefig('./DEM/Testrun_DEM_'+str(int(elapsed_time/out_int)).zfill(zp)+'.png')
        plt.close()
        plt.figure()
        imshow_grid(mg,mg.at_node['drainage_area'] > crit_area)
        plt.savefig('./dd/Testrun_dd_'+str(int(elapsed_time/out_int)).zfill(zp)+'.png')
        plt.close()
        #plt.figure()
        #imshow_grid(mg,fr.drainage_area,cmap='bone')
        #plt.savefig('./ACC/Testrun_ACC_'+str(int(elapsed_time/out_int)).zfill(zp)+'.png')
        #plt.close()
        #write_netcdf('./NC/output{}.nc'.format(str(ncCounter).zfill(zp)), mg, format='NETCDF4')
        
    #update elapsed time
    elapsed_time += dt
tE = time.time()
print()
print('Congratulations. Skript is done took {}s to run'.format(tE - t0))

/Users/mschmid/anaconda/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/mschmid/anaconda/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Elapsed Time: 0 writing output!


/Users/mschmid/landlab/landlab/components/drainage_density/drainage_density.py:204: UserWarning: Existing channel__mask grid field was overwritten.
  warn("Existing channel__mask grid field was overwritten.")


Elapsed Time: 100 writing output!
Elapsed Time: 200 writing output!
Elapsed Time: 300 writing output!
Elapsed Time: 400 writing output!
Elapsed Time: 500 writing output!
Elapsed Time: 600 writing output!
Elapsed Time: 700 writing output!
Elapsed Time: 800 writing output!
Elapsed Time: 900 writing output!
Elapsed Time: 1000 writing output!
Elapsed Time: 1100 writing output!
Elapsed Time: 1200 writing output!
Elapsed Time: 1300 writing output!
Elapsed Time: 1400 writing output!
Elapsed Time: 1500 writing output!
Elapsed Time: 1600 writing output!
Elapsed Time: 1700 writing output!
Elapsed Time: 1800 writing output!
Elapsed Time: 1900 writing output!
Elapsed Time: 2000 writing output!
Elapsed Time: 2100 writing output!
Elapsed Time: 2200 writing output!
Elapsed Time: 2300 writing output!
Elapsed Time: 2400 writing output!
Elapsed Time: 2500 writing output!
Elapsed Time: 2600 writing output!
Elapsed Time: 2700 writing output!
Elapsed Time: 2800 writing output!
Elapsed Time: 2900 writing ou

## PLOTS

### Hillslope Erosion / River Erosion

In [ ]:
fig, ax1 = plt.subplots(figsize = [11,7])
ax2 = ax1.twinx()
ax1.plot(timeVec, mean_hill_E, 'k', alpha = 0.6, linewidth = 2.5)
ax1.plot(timeVec, mean_riv_E, 'k--', alpha = 0.6, linewidth = 2.5)
#ax1.set_ylim([uplift_rate*0.9,uplift_rate*1.1])
ax1.plot(timeVec, mean_E, 'r', linewidth = 4.7)
ax2.plot(timeVec,100*vegi_test_timeseries,'g', linewidth = 4)
#ax2.set_ylim([0,100])

ax1.set_xlabel('years', fontsize = 22)
ax1.set_ylabel('Erosion rate', color='k', fontsize = 22)
ax2.set_ylabel('Vegetation cover [%]', color='k', fontsize = 22)
ax1.legend(['Hillslope Erosion','Fluvial Erosion', 'Total Erosion'], loc = 3, fontsize = 18)
ax2.legend(['Vegetation Cover'], loc = 4, fontsize = 18)
plt.savefig('./VegiEros_dualy.png',dpi = 720)
plt.show()

### Vegetation / Drainage Density

In [ ]:
fig, axarr = plt.subplots(5, sharex = True, figsize = [11,14])
axarr[0].plot(timeVec, vegi_test_timeseries,'g', linewidth = 2.5)
axarr[0].set_title('Mean Surface Vegetation', fontsize = 12)
axarr[0].set_ylabel('Vegetation cover')
axarr[1].plot(timeVec, mean_elev, 'k', linewidth = 2.5)
axarr[1].plot(timeVec, max_elev, 'k--', linewidth = 2, alpha = 0.5)
axarr[1].plot(timeVec, min_elev, 'k--', linewidth = 2, alpha = 0.5)
axarr[1].set_title('Mean Elevation', fontsize = 12)
axarr[1].set_ylabel('Mean Elevation [m]')
#axarr[1].set_ylim([0,80])
axarr[2].plot(timeVec, np.degrees(np.arctan(mean_slope)), 'r', linewidth = 2.5)
axarr[2].plot(timeVec, np.degrees(np.arctan(max_slope)), 'r--', linewidth = 2.0, alpha = 0.5)
axarr[2].plot(timeVec, np.degrees(np.arctan(min_slope)), 'r--', linewidth = 2.0, alpha = 0.5)
#axarr[2].set_ylim([0,10])
axarr[2].set_title('Mean Slope', fontsize = 12)
axarr[2].set_ylabel('Mean Slope [°]')
axarr[3].plot(timeVec,mean_dd, 'b', linewidth = 2.5)
axarr[3].set_title('Mean Drainage Density')
axarr[3].set_ylabel('Drainage Density')
axarr[4].plot(timeVec, mean_hill_E, 'g--', linewidth = 2.0, alpha = 0.5)
axarr[4].plot(timeVec, mean_riv_E, 'b--', linewidth = 2.0, alpha = 0.5)
axarr[4].plot(timeVec, mean_E, 'r--', linewidth = 2.2, alpha = 0.8)
axarr[4].legend(['Hillsl.', 'Rivers','Mean'])
axarr[4].set_title("Erosion rates")
axarr[4].set_ylabel('Erosion rate [m/yr]')
axarr[4].set_xlabel('Model Years', fontsize = 12)
plt.savefig('./Multiplot.png',dpi = 720)
plt.show()
#plt.close()

### Normalizing metrics

In [ ]:
mean_E_norm = mean_E/np.max(mean_E)
mean_riv_E_norm = mean_riv_E/np.max(mean_riv_E)
mean_hill_E_norm = mean_hill_E/np.max(mean_hill_E)
mean_elev_norm = mean_elev/np.max(mean_elev)
mean_slope_norm = mean_slope/np.max(mean_slope)
mean_dd_norm = mean_dd/np.max(mean_dd)

In [ ]:
fig, ax = plt.subplots(figsize = [11,7])
ax.plot(timeVec,vegi_test_timeseries, 'g--', linewidth = 5, alpha = 0.6)
#ax.plot(timeVec,mean_E_norm,'r--', linewidth = 5)
#ax.plot(timeVec,mean_elev_norm, 'k', linewidth = 5)
ax.plot(timeVec,mean_slope_norm, 'b', linewidth = 5, alpha = 0.7)
ax.plot(timeVec, mean_dd_norm, 'm', linewidth = 5)
#ax.legend(["Vegetation", "Erosion", "Elevation", "Slope", "Drainage Density"])

### Maps

In [ ]:
imshow_grid(mg,'topographic__steepest_slope')

In [ ]:
imshow_grid(mg,'topographic__elevation')

In [ ]:
imshow_grid(mg,'drainage_area')

In [ ]:
imshow_grid(mg,mg.at_node['drainage_area'] > crit_area)